# Jayden Jayawardhena

## Research question/interests

I am interested in which metal oxides were more present at given seasons throughout the year.

In [35]:
import pandas as pd

data = pd.read_csv("../data/raw/AirQuality.csv",sep=';')
data = data.drop(['Unnamed: 15','Unnamed: 16'],axis=1)
shape = data.shape
for column in data:
    print(data[column])
    




0       10/03/2004
1       10/03/2004
2       10/03/2004
3       10/03/2004
4       10/03/2004
           ...    
9466           NaN
9467           NaN
9468           NaN
9469           NaN
9470           NaN
Name: Date, Length: 9471, dtype: object
0       18.00.00
1       19.00.00
2       20.00.00
3       21.00.00
4       22.00.00
          ...   
9466         NaN
9467         NaN
9468         NaN
9469         NaN
9470         NaN
Name: Time, Length: 9471, dtype: object
0       2,6
1         2
2       2,2
3       2,2
4       1,6
       ... 
9466    NaN
9467    NaN
9468    NaN
9469    NaN
9470    NaN
Name: CO(GT), Length: 9471, dtype: object
0       1360.0
1       1292.0
2       1402.0
3       1376.0
4       1272.0
         ...  
9466       NaN
9467       NaN
9468       NaN
9469       NaN
9470       NaN
Name: PT08.S1(CO), Length: 9471, dtype: float64
0       150.0
1       112.0
2        88.0
3        80.0
4        51.0
        ...  
9466      NaN
9467      NaN
9468      NaN
9469      N